In [16]:
from collections import defaultdict

In [17]:
raw_read_1 = "/media/eternus1/nfs/projects/nkureev/mini_reads/amp_res_1.fastq"
raw_read_2 = "/media/eternus1/nfs/projects/nkureev/mini_reads/amp_res_2.fastq"

read_1_trimmomatic = "/media/eternus1/nfs/projects/nkureev/hiseq2000/trimmomatic/trimmomatic_hiseq2000_pairedPE_1.fq"
read_2_trimmomatic = "/media/eternus1/nfs/projects/nkureev/hiseq2000/trimmomatic/trimmomatic_hiseq2000_pairedPE_2.fq"

reads_1_v2trim = "/media/eternus1/nfs/projects/nkureev/mini_reads/v2trim/minireads.trim_1.fastq"
reads_2_v2trim = "/media/eternus1/nfs/projects/nkureev/mini_reads/v2trim/minireads.trim_2.fastq"

reads_1_fastp = "/media/eternus1/nfs/projects/nkureev/hiseq2000/fastp/fastp_hiseq2000_1.fq"
reads_2_fastp = "/media/eternus1/nfs/projects/nkureev/hiseq2000/fastp/fastp_hiseq2000_2.fq"

reads_1_trimgalore = "/media/eternus1/nfs/projects/nkureev/hiseq2000/trim_galore/trim_galore_hiseq2000/SRR1205228.1_2_val_1.fq"
reads_2_trimgalore = "/media/eternus1/nfs/projects/nkureev/hiseq2000/trim_galore/trim_galore_hiseq2000/SRR1205228.1_1_val_2.fq"

reads_1_prinseq = "/media/eternus1/nfs/projects/nkureev/hiseq2000/prinseq/prinseq_hiseq2000.fq_good_out_R1.fastq"
reads_2_prinseq = "/media/eternus1/nfs/projects/nkureev/hiseq2000/prinseq/prinseq_hiseq2000.fq_good_out_R2.fastq"

reads_1_adapterremoval = "/media/eternus1/nfs/projects/nkureev/hiseq2000/AdapterRemoval/AR_hiseq2000_1.fq"
reads_2_adapterremoval ="/media/eternus1/nfs/projects/nkureev/hiseq2000/AdapterRemoval/AR_hiseq2000_2.fq"

In [18]:
class FastqObj(object):

    def __init__(self, head, seq, strain, qual, phred33=False, forvard=True):
        self.head = head.strip()
        self.seq = seq.strip()
        self.qual = qual.strip()
        self.strain = strain.strip()
        if forvard:
            self.id = head.strip().replace("#", " ").split()[0].replace("@","") + "_1"
        else:
            self.id = head.strip().replace("#", " ").split()[0].replace("@","") + "_2"

In [19]:
def fastq_pe_reader(fastq_file1, fastq_file2):
    with open(fastq_file1) as fh1:
        with open(fastq_file2) as fh2:
            while True:
                try:
                    head1 = fh1.readline()
                    if head1 == '':
                        break
                    seq1 = fh1.readline()
                    strain1 = fh1.readline()
                    qual1 = fh1.readline()
                    fastq_obj1 = FastqObj(head1, seq1, strain1, qual1)

                    head2 = fh2.readline()
                    seq2 = fh2.readline()
                    strain2 = fh2.readline()
                    qual2 = fh2.readline()
                    
                    fastq_obj2 = FastqObj(head2, seq2, strain2, qual2, forvard=False)

                    yield fastq_obj1, fastq_obj2
                except:
                    break

In [20]:
raw_reads_gen = fastq_pe_reader(fastq_file1=raw_read_1, fastq_file2=raw_read_2)
v2trim_gen = fastq_pe_reader(fastq_file1=reads_1_v2trim, fastq_file2=reads_2_v2trim)
trimmomatic_gen = ""
fastp_gen = ""
trimgalore_gen = "" 
prinseq_gen = ""
adapterremoval_gen = "" 

raw_read_obj = (r for r in raw_reads_gen)
v2trim_read_obj = (r for r in v2trim_gen)
# trimmomatic_read_obj = (r for r in trimmomatic_gen)
# fastp_read_obj = (r for r in fastp_gen)
# trimgalore_read_obj = (r for r in trimgalore_gen)
# prinseq_read_obj = (r for r in prinseq_gen)
# adapterremoval_read_obj = (r for r in adapterremoval_gen)
# tools_to_compare = [v2trim_read_obj, trimmomatic_read_obj, fastp_read_obj, trimgalore_read_obj, prinseq_read_obj, adapterremoval_read_obj]

In [21]:
def reads_comparer(base_tool_obj, compared_tools_list):
    number_of_tools = len(compared_tools_list)
    while True:
        try:
            reads_to_compare = defaultdict(list)
            base_next = next(base_tool_obj)
            for base_read in base_next:
                base_id = base_read.id
                base_seq = base_read.seq
                reads_to_compare[base_id] = [0] * number_of_tools

            n_tool = 0
            for tool in compared_tools_list:
                tool_next = next(tool)
                for tool_read in tool_next:
                    tool_id = tool_read.id
                    tool_seq = tool_read.seq
                    if tool_id == base_id and tool_seq != base_seq:
                            reads_to_compare[base_id][n_tool] = 1
                n_tool += 1
            yield reads_to_compare
        except:
            break

In [36]:
raw_reads_gen = fastq_pe_reader(fastq_file1=raw_read_1, fastq_file2=raw_read_2)
v2trim_gen = fastq_pe_reader(fastq_file1=reads_1_v2trim, fastq_file2=reads_2_v2trim)
tools_list = [v2trim_gen]

compared_to_raw = reads_comparer(raw_reads_gen, tools_list)

i, j = 0, 0
for read in compared_to_raw:
    for read_id, changes in read.items():
        j += 1
        if changes[0] == 1:
            i += 1
print(j, i, i / j * 100)

911032 107 0.01174492224202882


In [62]:
different_reads = []
losted_reads = []
i = 0
for header, seq in trimmomatic_reads.items():
    if header in v2trim_reads.keys():
        if seq != v2trim_reads[header]:
            different_reads.append(header)
    else:
        losted_reads.append(header)

In [68]:
print(len(v2trim_reads[different_reads[0]]))
print(len(trimmomatic_reads[different_reads[0]]))

157
251


In [66]:
i = 0
for header, seq in v2trim_reads.items():
    if len(seq) == 251:
        i += 1

In [70]:
len(losted_reads)

4294

In [80]:
nontrimmed_by_trimmomatic = []
for key in different_reads:
    v2_seq = v2trim_reads[key]
    trim_seq = trimmomatic_reads[key]
    if len(v2_seq) > len(trim_seq):
        print(v2_seq.replace(trim_seq, ""))
    elif len(trim_seq) > len(v2_seq):
        nontrimmed_by_trimmomatic.append(trim_seq.replace(v2_seq, ""))

In [88]:
j = 0
for i in nontrimmed_by_trimmomatic:
    print(i)

AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAACCACTACAACATCACACTC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCTCATGCCGACTCCTGC
AGATCGGAAGAGCACACGGCTGAACTCCAGTCACGTAACATCATCTCGGCTGCCGTCTTCTGCTTGAAACACACAAAC
AGATCGGAAGAGCACACGGCTGAACTCCAGTCACGTAACATCACCTCGTATGCCGTCTTCTGCTTTAAAAAAAAAAATATATAACGCATCTATACAACTCATCTCACACCACACGACCACACAACACGATGACAGCTACATCTATC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTCGAAAAAAAACAACACACACACTCTACACGCAACACACGCTGCACA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAACAACAAACCACCACTACACGACAGTCTGCTCCGCGCGACACTGATACTCTGTCACAAGTAGTCTAGAGTGACAGGTCTAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAACAAAAAGTGACAAAAACAGCTATATGCACAATACATACACAACACAGACCACACAGAGACTCACGAGCACGTCACGT
AGATCGGAAGAGCACACGTCTGAACTCCAGGCACGCAACATCATCTCGTATGCCGACTTCTGCGTGAGAACACACGAGTACACTGTCC
GGCAGAGCACACGTCTGAACTCCAGCCAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAACAAATAGAAT

AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAACCAAACCCACACTGACAGCGAGCGACACACGCGCATGACTCACACGCCTGAGGCTAGCGATCGTCGATCGATACATC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTCACATCATCTCGTATGCCGTCTTCTGCCTGA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCACCTCGGATGCCGTCCTCTGCTTGCACAAAAAACGCGCAGTCACGTCTCCACACACACGACGTG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCCTGTATGACGTCCTCTGCTTGAATACAAACTAGAGACTACACACACAA
AGATCGGAAGAGCACACGTCTGAACTCCCGTCACGTAAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGACTTCTGCTTGAAAAAAAAAAAAATTTGGGCACCGTACTGATTCTTCTCTAGGGAAGACGCACGCGGGGACTGTAGACAAAGCGACATACTGACACACCA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAATACACTACCAGCGACACATAAGCGCACACACGATCTCACGCACAGAGGCG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCAGCTCGTATGCCGTCTGCTGCTTGAAAAAACAAGAATGTAGACCACATGGATCCTGATGTTGGAGCACGACGGGTGGCATTGCTCTATGCGGCCTTCCGTATCACTCTATGA
AGATCCGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCCGCTTGAAAAAAAAAAATACAACGCAATCGC

AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAATCTAGACGCGCGATAGCGATGAGCTATCGAGATATTCTCGCCACAGAAGAGCTCCGCTGCCGCG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAACAACACAACAGCACAGAGTCTACGACCACCACGTGGCTCACACCAGCGAGTATACAACACAGATACACATAAGATGACAACAGA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGACAAAAAAAAAACAACAACACAACTACACACTGCCTGCAACACCACATCTC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCCTGACAAAAAAGTAAGACGTCGCCACACCCTGGTTCTGTGCTCCAGTCCTGTGTTGTCGCGCGCGA
CGAGCACACGTCTGAACTCCAGTCACGCAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTACACACAACCCGGAGGCGCGCTGCTGCTTGACAACATAACAAAAGAGCGC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGCAACATCATCTCGTATGCCGTCTTCCGCTTGAAAAAAAACAAAACACCACAGACCTCTGGCTGTGCGACAGCTCACCACATGTGA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCT
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGGAACAT
CATCATCTCGTATGCCGTCTTCTGCTTGCAAAAAAAAAATA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCT

GGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGGAGCACACGTCTGCACTCCAGTCACAGGCAATGATCTCGTATGCCGTCCTCTGCTATGCGCACAAGTCG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAACAAAACTGACTCACAAACATGACGTCAACTATCTCTACCAGTCACGTAGAATCAGCACACATCGACAGCACCACATCACGAGCACGATACACACCTT
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCATGCAAAAAACAAATGTCTTGTGTCTTGTTTGACACTGCAGTTAAAGCTCCACACATGGCGATA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGCCTTCTGCTTGCAAAACAAAAA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAGCACTGAGTGCTAGCAGGCCTCACTACACACGCGTCACGCGAGAGTCACATATCGCAGTGTGATACAGCACCGGGACACCAGC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAACAACACGACAGTGACGCATCCACTCAGCGCAACACCGTCGCCGGCGCAGCACGT
GNNNNN
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGT
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGCAACATCATCTCG
AGATCGGAAGAGCACACGTCTGAAAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCCGCT

AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTCTGCCGTCT
NNNNNNNNNNNCGATGAGCGGTTTACCGCTATCTGGCAACCGAGCGAAGCGTGTGAACAGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCCTCTGCTTGAAAAAAATA
AGATCGGAAGAGCACACGTCTGAACTGCACGCACGTGACATCAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTATGCTTGATTATCTCTGTCACTTCGTCTCACACCACCTCTGTGA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAACAACACTCACCGACGTAGGAGAGACACGCGCGCAGACACACTCGCATACACGTCCAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAACACACAACCACGAGAACACAACCTCTATCACCGTAGACCTACACAGCACGCGACACCAGC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCCGCTTGACAAAA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAGAAAACGAGCTGAGAGTGCCACATGTCATG
AGAACGGAAGAGCACACGACTGAACTCCAGTCACGTAACATCCTCCCGTATGGCGTCTTCTGCCTCATAAAGACTAATACAACAGCGCGCCTCGTAGATACGAGATTTGTCTGTTCACACACCAGACACGACACCTGTGCTCGAACACGACAGCACACCACTACTCCGTG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAAC

AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAACACATAACACAGCGGCACACACATCAGCACCTCCGTTATATACTGTCGTGATACATCGCGCCAGTN
GGGGGGGGGGGGGGGCTCGGCAAATTGTAGACACAAAAGCCGCCGGCAGGGGTGTCGCCCACTCCCGGCATCTATCCGCGGCGCCGCCGACCGCAGTGCGAATATCTG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAACAACCAACACCAGCAGATGTTACACAGCCGTACACAACAGTCACACATCACTAGGATGTAACTCAGTACGAC
AGATCGGAAGAGCACACGTCTGAACTCCAGGCACGTAACATCAACTCGTATGCCGTCCTCTGCTTGCAAAAAAAAAAACACAAAACATCGAGCGAGCTGACTAGCACACACGTGCACTCGTCGCACGCAGATGCACAGCCTCAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAACAAAAGTAACAGAGACCTGCGCCAGATTCATGACGCCGACACACACAGCCAGATAGATAGTCTGCATCTCTCATCACCGCGTTGTATATCTG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGGAAAAAAAAAACACACACTACTGCCGACGCCAGCTGCTCGCGTAGCTCGCACGATCTGCACTAGATC
AGATCGGAAGAGCACACCGCTGCACTCCAGTCCCATAACAACATCGCCGAGCACGTCCTC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGGAACCTCCACTCGTATGCCGCCCTCTTCTTGAAAA
AGATCGGAAGAGCACACGTCTGA

AGATCGGAAGAGCACCCGTCTGAACTCCAGTCCCGTAACATCATCACGTATC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCCTCTCGTATGCCGTCCTCTGCTTGAAAAAAACACAAGACACACATG
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAACAAAAACAACACACGCGAGTATACGTACAA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAACTAAAGGAACCTGATTCAACGTTACATACCCAGACAGAGACACCCACACAGTGCAGCTCATACCACGAGCATGCTCGACATCTGCC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAACAAAAAAGCAACACTACATATGTGAGCGTACTGCACACGCACACACGCTCTGCTCTGCACAC
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGACAAAAAAAACAACAAA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAACACAAGACACATATCGAATCACATACCAAGATCAGCATACACACTCTACAA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCCTGACACTA
AGATCGGAAGAGCACACGTCTGAACTCCAGTCACGTAACATCATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAAGCACAATGTACACGAACACTCTCAG
AGATCGGAAGAGCACA

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
fastq_file1 = raw_read_1
fastq_file2 = raw_read_2

with open(fastq_file1) as fh1:
        with open(fastq_file2) as fh2:
            while True:
                head1 = fh1.readline()
                if not head1: break
                seq1 = fh1.readline()
                strain1 = fh1.readline()
                qual1 = fh1.readline()
                fastq_obj1 = FastqObj(head1, seq1, strain1, qual1)

                head2 = fh2.readline()
                seq2 = fh2.readline()
                strain2 = fh2.readline()
                qual2 = fh2.readline()
                fastq_obj2 = FastqObj(head2, seq2, strain2, qual2, forvard=False)

In [19]:
for i in fas

NameError: name 'fastq_obj1' is not defined

In [ ]:
## OLD STUFFF

In [ ]:
def create_last_table(reads_to_compare_table, reads_to_compare_dir, *compared_tools_reads_dirs):
    n_column = 1
    for tool in compared_tools_reads_dirs:
        reads_to_compare_table = between_tools_compare(reads_to_compare_table, reads_to_compare_dir, tool, n_column)
        n_column += 1
    return reads_to_compare_table

In [ ]:
# raw_reads compare with trimmomatic and v2_trim reads 0 - not equal, 1 - only one read equal, 2 - both reads equal
# raw_reads_compare[header] = [trimmomatic, v2trim]
def between_tools_compare(tool_to_compare_table, tool_to_compare_reads, compared_tool_reads, column_number):  
    
    for header, seq in tool_to_compare_reads.items():

        tc_forward = seq[0]
        tc_reverse = seq[1]

        if header in compared_tool_reads:
            ct_forward = compared_tool_reads[header][0]
            ct_reverse = compared_tool_reads[header][1]

            if tc_forward == ct_forward :
                if tc_reverse == ct_reverse:
                    tool_to_compare_table[header][column_number - 1] = 2
                else:
                    tool_to_compare_table[header][column_number - 1] = 1
            elif tc_reverse == ct_reverse:
                tool_to_compare_table[header][column_number - 1] = 1
    return tool_to_compare_table

In [3]:
def reads_reader(reads_file_1, reads_file_2):    
    reads = {}
    header = None
    with open(reads_file_1) as fh1:
        while True:
            header = fh1.readline().split(" ")[0][1:]
            if not header:
                break
            seq = fh1.readline()
            strand = fh1.readline()
            strand_seq = fh1.readline()
            reads[header] = [seq]
            
    with open(reads_file_2) as fh2:
        while True:
            header = fh2.readline().split(" ")[0][1:]
            if not header:
                break
            seq = fh2.readline()
            strand = fh2.readline()
            strand_seq = fh2.readline()
            reads[header].append(seq)
    return reads

In [ ]:
def create_compare_table(reads_dict, n_columns):
    reads_to_compare = {}
    for key in raw_reads.keys():
        reads_to_compare[key] = [0]*n_columns
    return reads_to_compare

In [ ]:
raw_reads_to_compare_table = create_compare_table(raw_reads, 6)

In [ ]:
rr_table = create_last_table(raw_reads_to_compare_table, raw_reads, trimmomatic_reads, v2trim_reads)

In [97]:
def reads_reader(reads_file_1, reads_file_2 ):    
    reads = {}
    header = None
    with open(reads_file_1) as fh1:
        for i, line in enumerate(fh1):
            line = line.strip()
            if line.startswith("@SRR"):
                if header:
                    reads[header] = seq
                header = line.split()[0][1:]
                seq = []
                header_switcher = True
            elif header_switcher:
                seq.append(line)
                header_switcher = False
    if header:
        reads[header] = seq
        
    with open(reads_file_2) as fh2:
        for i, line in enumerate(fh2):
            line = line.strip()
            if line.startswith("@SRR"):
                if header:
                    reads[header].append(seq)
                header = line.split()[0][1:]
                seq = ""
                header_switcher = True
            elif header_switcher:
                seq += line
                header_switcher = False
    if header:
        reads[header].append(seq)
    return reads